In [1]:
# ======================
# Imports
# ======================
import math as mt
import scipy.optimize as spo

# ======================
# Scenario Setup
# ======================
# Given data
S1 = 400e3          # Load 1 = 400 kW
pf1 = 0.85           # Power factor (lagging)
S2 = 300e3          # Load 2 = 300 kW
pf2 = 0.75           # Power factor (lagging)
Z_line = 1 + 2.5j    # Line impedance (ohms)
V_source = 13.8e3    # Source RMS voltage (volts)

# Compute power angles
theta1 = mt.acos(pf1)
theta2 = mt.acos(pf2)

# Complex powers for both loads (S = P + jQ)
S_load1 = S1 * (mt.cos(theta1) + 1j * mt.sin(theta1))
S_load2 = S2 * (mt.cos(theta2) + 1j * mt.sin(theta2))

# ======================
# Residual function definition
# ======================
def resid_from_x(V):
    """
    Residual function f(V) = |V + I*Z_line| - V_source
    where I = (S_total.conjugate()) / V
    """
    S_total = S_load1 + S_load2       # total complex power
    I = S_total.conjugate() / V       # current from source
    V_send = V + I * Z_line           # sending-end voltage
    return abs(V_send) - V_source     # residual to drive to zero

# ======================
# Bisection method setup
# ======================
XL = 10000      # lower bound of interval (10 kV)
XU = 13800      # upper bound of interval (13.8 kV)
MAX_ITERS = 750
X_TOL = 1e-7

# Run bisection method
p_bs, p_info = spo.bisect(
    resid_from_x, XL, XU,
    maxiter=MAX_ITERS, xtol=X_TOL, full_output=True
)

# ======================
# Results
# ======================
print(f"Finding a root in the interval [{XL}, {XU}]:")
print(p_info)
print(f"Residual value: {resid_from_x(p_bs)} V")


Finding a root in the interval [10000, 13800]:
      converged: True
           flag: converged
 function_calls: 38
     iterations: 36
           root: 13683.765423568548
         method: bisect
Residual value: -2.7452188078314066e-08 V
